In [ ]:
! pip install -r requirements.txt

In [5]:
import os
import json
import requests
import time

from openai import AzureOpenAI
from pathlib import Path
from typing import Optional
from IPython.display import clear_output

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [6]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_api_key = os.getenv("AZURE_OPENAI_KEY")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

bing_search_subscription_key = os.getenv("BING_SEARCH_SUBSCRIPTION_KEY")
bing_search_url = f"{os.getenv('BING_SEARCH_URL')}v7.0/search"

In [7]:
def bing_search(query: str) -> list:
    """
    Perform a bing search against the given query

    @param query: Search query
    @return: List of search results

    """
    headers = {"Ocp-Apim-Subscription-Key": bing_search_subscription_key}
    params = {"q": query, "textDecorations": False}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})

    return json.dumps(output)


# bing_search("who is the ceo of Goldman Sachs")

In [8]:

client = AzureOpenAI(api_key=aoai_api_key, api_version=api_version, azure_endpoint=azure_endpoint)

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

available_functions = {
    "search_bing": bing_search,
}

def get_tool_response(messages):
    
    current_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

    response_message = current_response.choices[0].message
    tool_calls = response_message.tool_calls
    messages.append(response_message)

    is_calling_function = True if tool_calls else False
    print(f"calling function: {is_calling_function}")

    while is_calling_function:

        tool_call = tool_calls[0]

        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(**function_args)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
        current_response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )
        response_message = current_response.choices[0].message
        # print(response_message)
        messages.append(response_message)
        tool_calls = response_message.tool_calls
        is_calling_function = True if tool_calls else False
    
    # print("messages:")
    # print(messages[-1].content)

    return messages[-1].content


In [9]:
instructions = """
You are an assistant designed to help people answer questions.
You have access to query the web using Bing Search.
You should call bing search whenever a question requires up to date information or could benefit from web data.
Always include the Citation when looking at bing search with URL.
Answer in nice Markdown format.
"""
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "Who is the CEO of Goldman Sachs?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
The current CEO of Goldman Sachs is David M. Solomon. He has held this position since October 2018 and also serves as the chairman of the bank since January 2019.

For more information, visit [David M. Solomon's Wikipedia page](https://en.wikipedia.org/wiki/David_M._Solomon) or [his profile on Goldman Sachs' official website](https://www.goldmansachs.com/our-firm/our-people-and-leadership/leadership/executive-officers/david-solomon).


In [10]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "Who is the COO of Goldman Sachs?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
The COO of Goldman Sachs is John Waldron.

Citation: [Goldman Sachs Leadership](https://www.goldmansachs.com/our-firm/our-people-and-leadership/leadership)


In [12]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "What is Goldman Sachs Net worth?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
The net worth of Goldman Sachs as of 2023 cannot be precisely determined from the snippets provided; however, one source mentions that Goldman Sachs has a market capitalization of $167.84 billion as of September 19, 2024. It's important to note that a company's net worth can be understood as its market capitalization, which is the current stock price multiplied by the number of shares outstanding.

For the most accurate and latest figures, please refer to [Goldman Sachs' official investor relations website](https://www.goldmansachs.com/investor-relations/financials/current/annual-reports/2023-annual-report/) or financial reporting platforms such as Macrotrends.

[Citation - Macrotrends](https://www.macrotrends.net/stocks/charts/GS/goldman-sachs/net-worth)


In [13]:
messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "What are some of the AI initive within Goldman Sachs?"},
]

response = get_tool_response(messages)
print(response)

calling function: True
Goldman Sachs has been actively investing in AI initiatives, aiming to leverage artificial intelligence to bolster their business operations, enhance customer experiences, and contribute to innovation within the financial industry. Some of the AI initiatives and perspectives within Goldman Sachs include:

- Research into the impact of generative AI on economic growth, with Goldman Sachs Research estimating that AI could contribute 1.5% to annual productivity growth over a ten-year period, potentially lifting global GDP by nearly $7 trillion. [Link](https://www.goldmansachs.com/insights/articles/the-generative-world-order-ai-geopolitics-and-power)
  
- Development of generative AI tools, with expectations from Goldman Sachs' Chief Information Officer Marco Argenti that the integration and adaptation of these tools will occur more quickly compared to previous technology waves. This also includes deploying generative AI tools for code generation to developers across